In [1]:
from crewai import Crew, Task, Agent
from dotenv import load_dotenv
import os
import yaml

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

/Users/ualguest/Desktop/Projects/CrewAI/venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [6]:
files = {
    'agents': '7_automated_project_config/agents.yaml',
    'tasks': '7_automated_project_config/tasks.yaml',

}

with open(files['agents'], 'r') as file:
    agents = yaml.safe_load(file)

with open(files['tasks'], 'r') as file:
    tasks = yaml.safe_load(file)



In [8]:
from pprint import pprint
pprint(agents)

{'estimation_agent': {'allow_delegation': False,
                      'backstory': 'You are the go-to expert for project '
                                   'estimation in {industry}. With a wealth of '
                                   'experience and access to vast historical '
                                   'data, you can predict the resources '
                                   'required for any task with remarkable '
                                   'accuracy. Your precision ensures that the '
                                   '{project_type} project remains feasible '
                                   'and avoids unnecessary delays or budget '
                                   'overruns.\n',
                      'goal': 'Provide highly accurate time, resource, and '
                              'effort estimations for each task in the '
                              '{project_type} project to ensure it is '
                              'delivered efficiently and o

In [9]:
pprint(tasks)

{'resource_allocation': {'description': 'Strategically allocate tasks for the '
                                        '{project_type} project to team '
                                        'members based on their skills, '
                                        'availability, and current workload. '
                                        'Ensure that each task is assigned to '
                                        'the most suitable team member and '
                                        'that the workload is evenly '
                                        'distributed.\n'
                                        '\n'
                                        'Team members:\n'
                                        '{team_members}\n',
                         'expected_output': 'A resource allocation chart '
                                            'showing which team members are '
                                            'responsible for each task in the '
            

In [10]:
#Creating Agents

project_planning_agent = Agent(
    config=agents['project_planning_agent']
)

estimation_agent = Agent(
    config=agents['estimation_agent']
)

resource_allocation_agent = Agent(
    config=agents['resource_allocation_agent']
)



In [12]:
#Creating Pydantic Objects for Outputs
from pydantic import BaseModel, Field
from typing import List

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="The name of the task")
    estimated_time_hours: str = Field(..., description="The estimated time to complete the task in hours")
    estimated_resources: str = Field(..., description="The estimated resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="The name of the milestone")
    tasks: List[str] = Field(..., description="The list of tasks IDs that are associated with this milestone")

class ProjectPlan(BaseModel):
    milestones: List[Milestone] = Field(..., description="The list of milestones for the project")
    tasks: List[TaskEstimate] = Field(..., description="The list of tasks with their estimates")



In [13]:
#Creating tasks
task_breakdown = Task(
    config=tasks['task_breakdown'],
    agent=project_planning_agent,
)

time_resource_estimation = Task(
    config=tasks['time_resource_estimation'],
    agent=estimation_agent,
)

resource_allocation = Task(
    config=tasks['resource_allocation'],
    agent=resource_allocation_agent,
    output_pydantic=ProjectPlan,
)

In [14]:
#Creating Crew

crew = Crew(
    agents = [project_planning_agent, estimation_agent, resource_allocation_agent],
    tasks = [task_breakdown, time_resource_estimation, resource_allocation],
    verbose=True,
)



In [15]:
project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

inputs = {
    'project_type': project,
    'industry': industry,
    'project_objectives': project_objectives,
    'team_members': team_members,
    'project_requirements': project_requirements,
}

result = crew.kickoff(inputs=inputs)


# Agent: The Ultimate Project Planner
## Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build tru

In [19]:
result.pydantic.model_dump()

{'milestones': [{'milestone_name': 'Design Phase',
   'tasks': ['Create a Responsive Design',
    'Implement a Modern User Interface',
    "Design 'Services' Page",
    "Create 'Contact Us' Page",
    'Include Testimonials Section']},
  {'milestone_name': 'Development Phase',
   'tasks': ['Develop User-Friendly Navigation System',
    "Create 'About Us' Page",
    'Implement Blog Section',
    'Optimize for Loading Speed and SEO',
    'Integrate Social Media Links']},
  {'milestone_name': 'Testing Phase',
   'tasks': ['Implement a Modern User Interface',
    'Develop User-Friendly Navigation System',
    'Optimize for Loading Speed and SEO']}],
 'tasks': [{'task_name': 'Create a Responsive Design',
   'estimated_time_hours': '40 hours',
   'estimated_resources': 'Bob Smith (Designer)'},
  {'task_name': 'Implement a Modern User Interface',
   'estimated_time_hours': '60 hours',
   'estimated_resources': 'Bob Smith (Designer), Alice Johnson (QA Engineer)'},
  {'task_name': 'Develop User-

In [20]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0010


/var/folders/3g/r3ht76p907vdqxh325xx85tm0000ks/T/ipykernel_37289/1074044196.py:7: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,6701,3560,0,3141,3


In [23]:
tasks = result.pydantic.model_dump()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,task_name,estimated_time_hours,estimated_resources
0,Create a Responsive Design,40 hours,Bob Smith (Designer)
1,Implement a Modern User Interface,60 hours,"Bob Smith (Designer), Alice Johnson (QA Engineer)"
2,Develop User-Friendly Navigation System,50 hours,"Jane Doe (Software Engineer), Tom Brown (QA Engineer)"
3,Create 'About Us' Page,30 hours,Jane Doe (Software Engineer)
4,Design 'Services' Page,50 hours,"Bob Smith (Designer), Jane Doe (Software Engineer)"
5,Create 'Contact Us' Page,40 hours,"Jane Doe (Software Engineer), Bob Smith (Designer)"
6,Implement Blog Section,50 hours,Jane Doe (Software Engineer)
7,Optimize for Loading Speed and SEO,40 hours,"Jane Doe (Software Engineer), Alice Johnson (QA Engineer)"
8,Integrate Social Media Links,30 hours,Jane Doe (Software Engineer)
9,Include Testimonials Section,40 hours,"Bob Smith (Designer), Jane Doe (Software Engineer)"


In [24]:
milestones = result.pydantic.model_dump()['milestones']
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,milestone_name,tasks
0,Design Phase,"['Create a Responsive Design', 'Implement a Modern User Interface', ""Design 'Services' Page"", ""Create 'Contact Us' Page"", 'Include Testimonials Section']"
1,Development Phase,"['Develop User-Friendly Navigation System', ""Create 'About Us' Page"", 'Implement Blog Section', 'Optimize for Loading Speed and SEO', 'Integrate Social Media Links']"
2,Testing Phase,"['Implement a Modern User Interface', 'Develop User-Friendly Navigation System', 'Optimize for Loading Speed and SEO']"
